In [196]:
from matplotlib import pyplot as plt
import numpy as np
import sympy as sp
import math
import pandas as pd

# Método da Bisseção

$\sqrt{x}-log_2{x}+2^{-x}$

In [197]:
x = sp.symbols("x")
y = sp.sqrt(x) - (sp.ln(x)/sp.ln(2)) + (2**(-x))

In [198]:
def encontrar_intervalos(funcao : sp.Function, variavel: sp.Symbol, dominio: list):

    f = sp.lambdify(variavel,funcao)

    inicio = dominio[0]
    fim = dominio[1]
    intervalos = []
    i = inicio + 1
    while i <= fim:
        if f(inicio)*f(i) <= 0:
            intervalos.append([inicio, i])
            inicio = i
        i = 2 * i
    return intervalos


def encontrar_raizes(funcao : sp.Function, variavel: sp.Symbol, x_min: float, x_max: float, epsilon: float):
    '''
        Dado uma função e um intervalo na quala função é contínua, ele utilizará o método da bisseção para encontrar suas raízes
    '''
    # Primeiro passo, converter em funções
    f = sp.lambdify(variavel, funcao)
    dy = sp.lambdify(variavel, funcao.diff())

    #Confirmar se f(a)*f(b)<0
    if f(x_min)*f(x_max) > 0:
        print("Não é possível determinar se há uma raíz nesse intervalo")
        return

    #Segundo passo, encontrar os intervalos
    tabela : list[tuple[float,float,float,str, str]] = []
    a = x_min
    b = x_max
    while b - a > epsilon:
        x_med = (a + b)/2
        dy_a = "+" if dy(a) >=0 else "-"
        dy_b = "+" if dy(b) >=0 else "-"
        tabela.append((x_med,f(x_med),b-a, dy_a, dy_b))
        if (f(a)*f(x_med)) < 0:
            b = x_med
        else:
            a = x_med
    display(pd.DataFrame(tabela, columns=["x","f(x)","b - a", "derivada de a", "derivada de b"]))
    return x_med

def bissecao(funcao : sp.Function, variavel: sp.Symbol, dominio: list[float], epsilon: float):
    intervalos = encontrar_intervalos(funcao, variavel, dominio)
    raizes = []
    for inter in intervalos:
        print(f"INTERVALO [{inter[0]}, {inter[1]}]")
        x = encontrar_raizes(funcao, variavel, inter[0],inter[1],epsilon)
        print(f"x = {x}\n\n")
        raizes.append(x)
    return raizes

In [199]:
bissecao(y,x,[0.1,1000],10e-5)

INTERVALO [0.1, 8.8]


,x,f(x),b - a,derivada de a,derivada de b
0,4.450000,0.001450,8.700000,-,+
1,6.625000,-0.143881,4.350000,-,+
2,5.537500,-0.094515,2.175000,-,-
3,4.993750,-0.054068,1.087500,-,-
4,4.721875,-0.028478,0.543750,-,-
5,4.585938,-0.014098,0.271875,-,-
6,4.517969,-0.006476,0.135937,-,-
7,4.483984,-0.002552,0.067969,-,-
8,4.466992,-0.000561,0.033984,-,-
9,4.458496,0.000442,0.016992,-,-


x = 4.462279510498048


INTERVALO [8.8, 17.6]


,x,f(x),b - a,derivada de a,derivada de b
0,13.200000,-8.917933e-02,8.800000,+,+
1,15.400000,-2.055194e-02,4.400000,+,+
2,16.500000,1.763587e-02,2.200000,+,+
3,15.950000,-1.723612e-03,1.100000,+,+
4,16.225000,7.893294e-03,0.550000,+,+
5,16.087500,3.068705e-03,0.275000,+,+
6,16.018750,6.684571e-04,0.137500,+,+
7,15.984375,-5.286069e-04,0.068750,+,+
8,16.001562,6.966870e-05,0.034375,+,+
9,15.992969,-2.295333e-04,0.017187,+,+


x = 15.999615478515624




[4.462279510498048, 15.999615478515624]

# Converter número binário em ponto flutuante em número inteiro

$$00111110010100000000000000000000$$
$$01000001011100000000000000000000$$

In [209]:
def seq_to_ponto_flutuante(seq: str) -> str:
    # Separar a sequência em 3 partes
    sinal = seq[0]
    expoente = seq[1:9]
    mantissa = seq[9:]
    bias = 127

    # Transformar o sinal
    s = "-" if (-1)**int(sinal) == -1 else "+"

    #Transformar o expoente
    i = len(expoente) - 1
    exp_int = 0
    for binary in expoente:
        exp_int += int(binary) * pow(2,i)
        i -= 1

    # Cortar os zeros desnecessários da mantissa
    i = len(mantissa) - 1
    while i > 0:
        if (mantissa[i] == "1"):
            break
        i -= 1
    m = mantissa[:i+1]

    return f"{s}1.{m} × 2^{exp_int - bias}"

In [237]:
def ponto_flutuante_to_decimal(pf: str) -> float:
    sinal = pf[0]
    mantissa, expoente = [x.strip() for x in pf[1:].split("×")]

    #Calcular o expoente
    exp_int = int(expoente[2:])
    
    # Transforma a mantissa
    mantissa = mantissa[2:]
    i = 1
    mant_decimal = 0.0
    for bina in mantissa:
        mant_decimal += int(bina)*pow(2,-i)
        i += 1
    
    #Transformar o sinal
    s = (-1) if sinal == "-" else 1

    return s * (1+mant_decimal) * pow(2,exp_int)

In [503]:
def seq_to_decimal(seq: str):
    return ponto_flutuante_to_decimal(seq_to_ponto_flutuante(seq))

In [238]:
a = seq_to_ponto_flutuante("00111110010100000000000000000000")
print(a)
ponto_flutuante_to_decimal(a)

+1.101 × 2^-3


0.203125

In [504]:
seq_to_decimal("00111110010100000000000000000000")

0.203125

In [239]:
b = seq_to_ponto_flutuante("01000001011100000000000000000000")
print(b)
ponto_flutuante_to_decimal(b)

+1.111 × 2^3


15.0

In [505]:
seq_to_decimal("01000001011100000000000000000000")

15.0

# Somar binário e converter

In [485]:
def soma_bin_unit(bit1,bit2,over):
    count_1 = [bit1, bit2, over].count("1")
    match (count_1):
        case 0:
            return "0","0"
        case 1:
            return "1","0"
        case 2:
            return "0","1"
        case 3:
            return "1","1"
        case _:
            raise ValueError

In [486]:
def sub_bin_unit(bit1,bit2,over):
    count_1 = [bit2, over].count("1")
    if bit1 == "1":
        match (count_1):
            case 0:
                return "1","0"
            case 1:
                return "0","0"
            case 2:
                return "1","1"
    elif bit1 == "0":
        match (count_1):
            case 0:
                return "0","0"
            case 1:
                return "1","1"
            case 2:
                return "0","1"
    raise ValueError(f"{bit1}, {bit2}, {over}")

In [487]:
def soma_bin(bin1,bin2, sinal):
    if len(bin1) > len(bin2):
        bin2 += "0"* (len(bin1)-len(bin2))
    elif len(bin2) > len(bin1):
        bin1 += "0"* (len(bin2)-len(bin1))
    
    funcao = soma_bin_unit if sinal == "+" else sub_bin_unit

    result = ""
    over = "0"
    for i in range(len(bin1)-1,-1,-1):
        if bin1[i] == ".":
            result = "." + result
        else:
            bit_i, over = funcao(bin1[i],bin2[i],over)
            result = bit_i + result
    result = result if over == "0" else over + result
    
    if sinal == "-":
        novo_re = ""
        for i in range(len(result)):
            if result[i] == "1":
                novo_re += "0"
            elif result[i] == "0":
                novo_re += "1"
            else:
                novo_re += "."
        return soma_bin(novo_re, "0." + "0"*(len(result)-3) + "1", "+")

    return result

In [495]:
def soma_ponto_flutuante(bin1 : str, bin2):

    partes_1 = [bin1[0], bin1[1:bin1.index("×")].strip(), bin1[bin1.index("×")+1:].strip()]
    partes_2 = [bin2[0], bin2[1:bin2.index("×")].strip(), bin2[bin2.index("×")+1:].strip()]

    # Transformar o expoente de ambas
    partes_1[2] = int(partes_1[2][2:])
    partes_2[2] = int(partes_2[2][2:])

    # Mantenho o sinal da maior mantissa
    sinal = partes_1[0] if partes_1[2] > partes_2[2] else partes_2[0]

    #defino qual o maior expoente e igualo os números
    if (partes_2[2] != partes_1[2]):
        temp = partes_1 if partes_1[2] < partes_2[2] else partes_2
        novo_exp = max(partes_1[2], partes_2[2])
        temp[1] = temp[1].split(".")
        temp[1][0] = "0." + "0"*(novo_exp-temp[2]-1)+temp[1][0]
        temp[1] = "".join(temp[1])
        temp[2] = novo_exp

    
    # Retiro o expoente em comum
    expoente = partes_1[2]

    # Somar as mantissar
    mantissa = soma_bin(partes_1[1],partes_2[1], sinal)

    # Normalizar o resultado
    m = mantissa.split(".")
    expoente += len(m[0]) - 1
    mantissa = m[0][0] + "." + m[0][1:] + m[1]

    return ponto_flutuante_to_decimal(f"{sinal}{mantissa} × 2^{expoente}")

In [499]:
print(ponto_flutuante_to_decimal("+1.101 × 2^2"))
print(ponto_flutuante_to_decimal("+1.111 × 2^3"))
soma_ponto_flutuante("+1.101 × 2^2", "+1.111 × 2^3")

6.5
15.0


21.5

# Converter binários e somar

In [506]:
def converter_e_soma(bin1,bin2):
    dec1 = ponto_flutuante_to_decimal(bin1)
    dec2 = ponto_flutuante_to_decimal(bin2)
    return dec1 + dec2

In [507]:
converter_e_soma("+1.101 × 2^2", "+1.111 × 2^3")

21.5